# 0) Import de l'API Github

In [1]:
!git clone https://github.com/dbeley/senscritiquescraper
%cd senscritiquescraper
!python setup.py install
!python examples/scr_get_work_urls.py -h

Cloning into 'senscritiquescraper'...
remote: Enumerating objects: 598, done.
remote: Counting objects: 100% (262/262), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 598 (delta 162), reused 164 (delta 81), pack-reused 336
Receiving objects: 100% (598/598), 157.04 KiB | 4.49 MiB/s, done.
Resolving deltas: 100% (382/382), done.
/content/senscritiquescraper
running install
running bdist_egg
running egg_info
creating senscritiquescraper.egg-info
writing senscritiquescraper.egg-info/PKG-INFO
writing dependency_links to senscritiquescraper.egg-info/dependency_links.txt
writing requirements to senscritiquescraper.egg-info/requires.txt
writing top-level names to senscritiquescraper.egg-info/top_level.txt
writing manifest file 'senscritiquescraper.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'senscritiquescraper.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
c

# 1) Import des packages

In [2]:
import pandas as pd
from senscritiquescraper import Senscritique
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time

# 2) Connexion a Google drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
%cd '/gdrive/My Drive/Web_scraping'

/gdrive/My Drive/Web_scraping


In [ ]:
%ls

 senscritiquescraper/
'Test API web scraping'
 tracks_features.csv
 tracks_features_max_popu+YT_0_10000.csv
 tracks_features_max_popu+YT_0_12400.csv
 tracks_features_max_popu+YT_0_3000.csv
 tracks_features_max_popu+YT_0_4000.csv
 tracks_features_max_popu+YT_0_8000.csv
 tracks_features_max_popu+YT_0_9000.csv
 tracks_features_popu_100_pourcents.csv
 tracks_features_vues_0_10000.csv
 tracks_features_vues_0_1000.csv
 tracks_features_vues_0_2000.csv
 tracks_features_vues_0_5000.csv
 tracks_features_vues_0_7000.csv
 tracks_features_vues_100.csv


# 3) Web Scraping Sens Critique

In [ ]:
debut, fin = 0,1000

In [ ]:
df_reduit = pd.read_csv("tracks_features_vues_0_" + str(debut) + ".csv", index_col = 0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
def get_titre_artiste_note(url):
  L_resultat = []

  page_SC = urlopen(url)
  soup = BeautifulSoup(page_SC, 'html.parser')
  titre_SC = soup.findAll(name = 'h1', attrs = {"class":"pvi-product-title "})
  artiste_SC = soup.findAll(name = 'span', attrs = {"itemprop":"name"})
  note_SC = soup.findAll(name = 'span', attrs = {"itemprop":"ratingValue"})
  
  if titre_SC != []: L_resultat.append(titre_SC[0].text.strip())
  else: L_resultat.append('')
  if artiste_SC != []: L_resultat.append(artiste_SC[0].text)
  else: L_resultat.append('')
  if note_SC != []: L_resultat.append(note_SC[0].text)
  else: L_resultat.append(0)

  return tuple(L_resultat)

#TEST
get_titre_artiste_note('https://www.senscritique.com/morceau/San/28139401')

('San', 'OrelSan', '8.1')

In [ ]:
L_note = []

t1 = time.time()

for i in range(1000):
  track_id = df_reduit.index[i]

  track_name = df_reduit.loc[track_id, 'name']
  artist_name = df_reduit.loc[track_id, 'artists'].split("'")[1]


  track_url = Senscritique.get_url(track_name, rank=1, genre="Morceau")

  if track_url == None:
    L_note.append(0)
  elif track_url.split('/')[3] == 'morceau' :
    titre_SC, artiste_SC, note_SC = get_titre_artiste_note(track_url)
    if track_name.lower() == titre_SC.lower() and \
        artist_name.lower() == artiste_SC.lower():
      L_note.append(note_SC)
    else:
      L_note.append(0)
  else:
    L_note.append(0)

t2 = time.time()

print('Durée :', t2-t1, '\n')

print(len(L_note),'\n', L_note)

Desired result not found in search results (Desired result: position 1, number of search results: 0).
Desired result not found in search results (Desired result: position 1, number of search results: 0).
Desired result not found in search results (Desired result: position 1, number of search results: 0).
Desired result not found in search results (Desired result: position 1, number of search results: 0).
Desired result not found in search results (Desired result: position 1, number of search results: 0).
Desired result not found in search results (Desired result: position 1, number of search results: 0).
Desired result not found in search results (Desired result: position 1, number of search results: 0).
Desired result not found in search results (Desired result: position 1, number of search results: 0).
Desired result not found in search results (Desired result: position 1, number of search results: 0).


Durée : 1568.6545462608337 

1000 
 ['8.0', '8.2', '8.1', '7.0', '8.4', '7.9', '6.7', 0, '7.2', '6.9', '7.1', '7.7', '8.2', '8.4', '8.1', '7.4', '8.0', '8.1', '7.8', '7.5', '7.4', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '', 0, 0, '', '7.7', 0, 0, 0, 0, '', '', '7.4', '', 0, '7.7', '', 0, '', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '', 0, 0, '', '', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '', '', '', 0, 0, 0, 0, 0, 0, 0, 0, '', 0, 0, '', 0, 0, 0, 0, 0, '', 0, 0, 0, 0, 0, 0, '', 0, 0, 0, 0, 0, 0, 0, '', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '', 0, 0, 0, 0, 0, 0, '', 0, '', '', 0, '', '', '', '', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '', '', 0, 0, 0, 0, 0, 0, 0, '', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '', 0, 0, 0, 0, 0, 0, 0,

In [ ]:
df_reduit_1000 = df_reduit.head(1000)
df_reduit_1000['note_SC'] = L_note
df_reduit_1000.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,name,album,artists,note_SC
id,,,,
7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,['Rage Against The Machine'],8.0
1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,['Rage Against The Machine'],8.2
1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,The Battle Of Los Angeles,['Rage Against The Machine'],8.1
2lbASgTSoDO7MTuLAXlTW0,Mic Check,The Battle Of Los Angeles,['Rage Against The Machine'],7.0
1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,The Battle Of Los Angeles,['Rage Against The Machine'],8.4


In [ ]:
df.to_csv("tracks_features_vues_0_" + str(fin) + ".csv")

In [ ]:
n_tot = df_reduit_1000.shape[0]
n_zero = df_reduit_1000[df_reduit_1000['note_SC'] == 0].shape[0]

print(100*n_zero/n_tot, "% des musiques ont des notes de '0' (soit parce qu'elle ne sont site ou juste parce qu'elle ne sont pas notées).\
                        \nJ'estime que ce site n'est un bon site pour notre projet car on perd beaucoup de données.")

83.5 % des musiques ont des notes de '0' (soit parce qu'elle ne sont site ou juste parce qu'elle ne sont pas notées).                        
J'estime que ce site n'est un bon site pour notre projet car on perd beaucoup de données.


**=> Changement de site !**

# 4) Web Scraping Youtube

On souhaite récupérer les vues et les likes sur nos musiques. Pour éviter de faire ce processus sur l'ensemble de nos données, on effectue ce traitement uniquement pour nos musiques les plus connue. 

In [ ]:
df_popu = pd.read_csv("tracks_features_popu_100_pourcents.csv", index_col = 0)
df_max_popu = df_popu[df_popu['ppopularity'] > 50]
print(df_max_popu.shape)
df_max_popu.to_csv('tracks_features_max_popu+YT_0_0.csv')

In [ ]:
debut, fin = 0,12436

df = pd.read_csv("tracks_features_max_popu+YT_0_" + str(debut) + ".csv", index_col = 0)
df.head(2)

,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date,popularity,views_YT,likes_YT
id,,,,,,,,,,,,,,,,,,,,,,,,,,
7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],1,1,False,0.470,0.978,7,-5.399,1,0.0727,0.0261,0.000011,0.356,0.503,117.906,210133,4.0,1999,1999-11-02,65,17375832,132343
1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],2,1,True,0.599,0.957,11,-5.764,1,0.1880,0.0129,0.000071,0.155,0.489,103.680,206200,4.0,1999,1999-11-02,69,7325119,100441


In [ ]:
from googlesearch import search
import re

def get_vues_likes(url): #0.5s environ
  page_YT = urlopen(url)
  txt = str(page_YT.read())

  nb_vues  = re.findall(r'"viewCount":"[0-9]*?"', txt)
  nb_likes = re.findall(r'"label":"[0-9,]*? likes', txt)

  if (len(nb_vues) == 0) or (len(nb_likes) == 0): #Important si la vidéo n'existe plus
    return 0, 0
  else : 
    nb_vues, nb_likes = nb_vues[0], nb_likes[0][:-6].replace(',','')
    return int(nb_vues.split('"')[3]), int(nb_likes.split('"')[3])

#TEST
get_vues_likes('https://www.youtube.com/watch?v=bWXazVhlyxQ')

(170839999, 1200461)

In [ ]:
L_vues = []
L_likes = []

for i in range(debut,fin):
  track_id = df.index[i]

  track_name = df.loc[track_id, 'name']
  artist_name = df.loc[track_id, 'artists'].split("'")[1]

  try:
    track_url = list(search(artist_name + ' ' + track_name,
                            tld='com',   
                            start=0,    
                            stop=1,
                            pause=2,
                            )
                    )

    if len(track_url) == 0:
      L_vues.append(0)
      L_likes.append(0)

    elif track_url[0].split('/')[2] != 'www.youtube.com' :
      L_vues.append(0)
      L_likes.append(0)

    else:
      vues, likes = get_vues_likes(track_url[0])
      L_vues.append(vues)
      L_likes.append(likes)

  except :
    print('HTTPError détectée mais ça stop pas')
    L_vues.append(-1)    #On gere l'erreur et on la traitera dans un second temps
    L_likes.append(-1)

  print(i, 'vues :', L_vues[-1], '// likes :', L_likes[-1])

12430 vues : 19563803 // likes : 50304
12431 vues : 0 // likes : 0
12432 vues : 67869252 // likes : 78346
12433 vues : 40490831 // likes : 78984
12434 vues : 0 // likes : 0
12435 vues : 5634848 // likes : 12865


In [ ]:
df_debut_fin = df.iloc[debut:fin]
df_debut_fin['views_YT'] = L_vues
df_debut_fin['likes_YT'] = L_likes

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
df.iloc[debut:fin] = df_debut_fin
df[['name', 'artists', 'views_YT', 'likes_YT']].iloc[fin-3:fin]

,name,artists,views_YT,likes_YT
id,,,,
7kjEqI6eZ7YlFrGhxixnlN,ANTES QUE SE ACABE,['Bad Bunny'],40490831,78984
2Iv6YBslkxslMJQsxshZh5,CANTARES DE NAVIDAD,['Trio Vegabajeño'],0,0
55QIntmvDd7pYpNKl5AK4s,In The Morning,['Jennifer Lopez'],5634848,12865


In [ ]:
df.to_csv('tracks_features_max_popu+YT_0_' + str(fin) + '.csv')